In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from utils.nn.sequential import Sequential, EarlyStopping
from utils.nn.layer import Dense, Dropout
from utils.nn.neuron import Activation
from utils.nn.optimizer import Adam
from utils.nn.losses import get_loss_function
from utils.nn.initializer import Initializer, InitializationType

# Universal Function Approximation

In [ ]:
def build_and_train_approximator(f, model, optimizer):

    """Builds, trains, and evaluates a function approximator model."""

    # 1. Generate training data
    x_values = np.linspace(0.1, 15, 200).reshape(-1, 1)
    noise = np.random.normal(0, 0.05, x_values.shape)
    y_values = f(x_values) + noise

    loss_function = get_loss_function('mse')    
    early_stopping = EarlyStopping(patience=20, min_delta=0.0001, restore_best_weights=True)

    model.fit(
        x_train=x_values,
        y_train=y_values,
        optimizer=optimizer,
        loss_func=loss_function,
        epochs=1000,
        batch_size=64,
        metric="r2_score",
        early_stopping=early_stopping,
        display_interval=50
    )

    # 4. Evaluate and visualize the approximation
    x_test = np.linspace(0.1, 15, 1000).reshape(-1, 1)
    y_true = f(x_test)
    y_pred = model.predict(x_test)
    
    r2 = model.evaluate(x_test, y_true, metric_name="r2_score")[1]
    print(f"\nR2 Score on test data: {r2:.4f}")
    plt.figure(figsize=(12, 8))

    # Plot the original true function
    sns.lineplot(x=x_test.flatten(), y=y_true.flatten(), color='blue', linewidth=3, label='True Function', zorder=10)

    # Plot the noisy data points as a scatter plot
    sns.scatterplot(x=x_values.flatten(), y=y_values.flatten(), color='red', alpha=0.5, label='Noisy Training Data', zorder=1)

    # Plot the model's predictions
    sns.lineplot(x=x_test.flatten(), y=y_pred.flatten(), color='green', linewidth=3, linestyle='--', label='MLP Approximation', zorder=5)

    plt.title('Function Approximation using a Multilayer Perceptron', fontsize=18)
    plt.xlabel('x', fontsize=14)
    plt.ylabel('y', fontsize=14)
    plt.legend(fontsize=12)
    plt.show()
    
    return x_test, y_true, y_pred

## Approximate Function 1

$f(x) = -0.03 \cdot x^2 \cdot 0.05 \cdot \ln(x^3) + 0.5 \cdot \cos\left(\frac{x}{1.5}\right) + 0.1 \cdot \sin(5x)$


In [ ]:
def complicated_function(x):
    return -0.03 * x**2  * 0.05 * np.log(x ** 3) + 0.5 * np.cos(x/1.5) + 0.1 * np.sin(x*5)


model = Sequential([
	Dense(shape=(1, 64), activation=Activation.SIGMOID, initializer=Initializer(InitializationType.GLOROT_UNIFORM, fan_in=1, fan_out=64)),
	Dense(shape=(64, 64), activation=Activation.SIGMOID, initializer=Initializer(InitializationType.GLOROT_UNIFORM, fan_in=64, fan_out=64)),
	Dense(shape=(64, 1), activation=Activation.LINEAR, initializer=Initializer(InitializationType.GLOROT_UNIFORM, fan_in=64, fan_out=1))
])

optimizer = Adam(params=model.parameters(), learning_rate=0.001)

build_and_train_approximator(complicated_function, model, optimizer)

## Approximate Function 2

$f(x) = e^{-0.5x}\sin(2x)$

In [ ]:
def complicated_function(x):
    """
    A function that represents a dampened wave.
    """
    return np.exp(-0.5 * x) * np.sin(x * 2)

model = Sequential([
	Dense(
		shape=(1, 32),
		activation=Activation.TANH,
		initializer=Initializer(fill_type=InitializationType.GLOROT_UNIFORM, fan_in=1, fan_out=32)
	),
	Dense(
		shape=(32, 16),
		activation=Activation.TANH,
		initializer=Initializer(fill_type=InitializationType.GLOROT_UNIFORM, fan_in=32, fan_out=16)
	),
	Dense(
		shape=(16, 16),
		activation=Activation.RELU,
		initializer=Initializer(fill_type=InitializationType.GLOROT_UNIFORM, fan_in=16, fan_out=16)
	),
	Dense(
		shape=(16, 1),
		activation=Activation.LINEAR,
		initializer=Initializer(fill_type=InitializationType.GLOROT_UNIFORM, fan_in=16, fan_out=1)
	)
])

optimizer = Adam(params=model.parameters(), learning_rate=0.001)

build_and_train_approximator(complicated_function, model, optimizer)